### Charles Vin & Aymeric Delefosse

#### M2-DAC


## Expérimentations
### Stabilité
Pour mesurer la variabilité due à la composante aléatoire de l'algorithme, on générera des exemples contrefactueles multiples pour la même observation d'entrée. On pourra également comparer la stabilité de l'algorithme en utilisant différents classifieurs.


